In [1]:
!curl -o 'amazon_reviews_us_Electronics_v1_00.tsv.gz' 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  666M  100  666M    0     0  63.3M      0  0:00:10  0:00:10 --:--:-- 65.9M


In [2]:
!pip install elasticsearch

     |████████████████████████████████| 92kB 5.9MB/s 


In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch

es = Elasticsearch()

In [2]:
import pandas as pd
df = pd.read_csv('amazon_reviews_us_Electronics_v1_00.tsv.gz', delimiter='\t',error_bad_lines=False)
df.head()

b'Skipping line 9076: expected 15 fields, saw 22\nSkipping line 19256: expected 15 fields, saw 22\nSkipping line 24313: expected 15 fields, saw 22\nSkipping line 47211: expected 15 fields, saw 22\nSkipping line 54295: expected 15 fields, saw 22\nSkipping line 56641: expected 15 fields, saw 22\nSkipping line 63067: expected 15 fields, saw 22\n'
b'Skipping line 93796: expected 15 fields, saw 22\n'
b'Skipping line 132806: expected 15 fields, saw 22\nSkipping line 164631: expected 15 fields, saw 22\nSkipping line 167019: expected 15 fields, saw 22\nSkipping line 167212: expected 15 fields, saw 22\n'
b'Skipping line 198103: expected 15 fields, saw 22\nSkipping line 199191: expected 15 fields, saw 22\nSkipping line 202841: expected 15 fields, saw 22\nSkipping line 218228: expected 15 fields, saw 22\nSkipping line 235900: expected 15 fields, saw 22\n'
b'Skipping line 277761: expected 15 fields, saw 22\nSkipping line 304582: expected 15 fields, saw 22\nSkipping line 312029: expected 15 fields,

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,41409413,R2MTG1GCZLR2DK,B00428R89M,112201306,yoomall 5M Antenna WIFI RP-SMA Female to Male ...,Electronics,5,0,0,N,Y,Five Stars,As described.,2015-08-31
1,US,49668221,R2HBOEM8LE9928,B000068O48,734576678,"Hosa GPM-103 3.5mm TRS to 1/4"" TRS Adaptor",Electronics,5,0,0,N,Y,It works as advertising.,It works as advertising.,2015-08-31
2,US,12338275,R1P4RW1R9FDPEE,B000GGKOG8,614448099,Channel Master Titan 2 Antenna Preamplifier,Electronics,5,1,1,N,Y,Five Stars,Works pissa,2015-08-31
3,US,38487968,R1EBPM82ENI67M,B000NU4OTA,72265257,LIMTECH Wall charger + USB Hotsync & Charging ...,Electronics,1,0,0,N,Y,One Star,Did not work at all.,2015-08-31
4,US,23732619,R372S58V6D11AT,B00JOQIO6S,308169188,Skullcandy Air Raid Portable Bluetooth Speaker,Electronics,5,1,1,N,Y,Overall pleased with the item,Works well. Bass is somewhat lacking but is pr...,2015-08-31


In [0]:
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch()
start = time.time()
with open('2014-07-23_18-11-16.json', "r", encoding = "utf-8") as read_file:
#     print('total post:',i)
    for i,j in enumerate(read_file):
#         print(j)
        res = es.index(index="tweets-index-1", doc_type='tweet', id=i, body=j)
#         print(res['result'])
end = time.time()
diff = end - start
print("It took %d on data processing" % (diff))

In [3]:
df1=df.copy()

In [4]:
df1 = df1[df1.review_date.notnull()]
df1 = df1[df1.product_title.notnull()]
df1 = df1[df1.product_id.notnull()]
df1 = df1[df1.customer_id.notnull()]

In [5]:
df1.count()

marketplace          3090996
customer_id          3090996
review_id            3090996
product_id           3090996
product_parent       3090996
product_title        3090996
product_category     3090996
star_rating          3090996
helpful_votes        3090996
total_votes          3090996
vine                 3090996
verified_purchase    3090996
review_headline      3090965
review_body          3090908
review_date          3090996
dtype: int64

In [8]:
!pip install nltk
!pip install matplotlib
# pip install --user -U nltk

In [6]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.svm import SVC

import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
import numpy as np

tokenizer = RegexpTokenizer('\s+', gaps=True)
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def canonicalize(string):
    normalized_tokens = list()
    tokens = tokenizer.tokenize(string)
    for token in tokens:
        if token.lower() not in stop_words and len(token) > 1 and len(token) < 24:
            normalized = stemmer.stem(token.lower())
            normalized_tokens.append(normalized)
    return normalized_tokens

def canonicalize_withoutstem(string):
    normalized_tokens = list()
    tokens = tokenizer.tokenize(string)
    for token in tokens:
        if token.lower() not in stop_words and len(token) > 1 and len(token) < 24:
            normalized = token.lower()
            normalized_tokens.append(normalized)
    return normalized_tokens

def eval_summary(name, predictions, labels):
    precision = precision_score(predictions, labels,average='macro')
    recall = recall_score(predictions, labels,average='macro')
    accuracy = accuracy_score(predictions, labels)
    f1 = fbeta_score(predictions, labels, 1,average='macro') #1 means f_1 measure
    print('\nConfusion matrix:\n',confusion_matrix(labels, predictions))
    print("Classifier '%s' has P=%0.3f R=%0.3f Acc=%0.3f F1=%0.3f" % (name,precision,recall,accuracy,f1) )
    print(classification_report(predictions, labels))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
for i in range(30):
    print(df.product_title[i],canonicalize_withoutstem(df1.product_title[i]))

yoomall 5M Antenna WIFI RP-SMA Female to Male Extensionl Cable ['yoomall', '5m', 'antenna', 'wifi', 'rp-sma', 'female', 'male', 'extensionl', 'cable']
Hosa GPM-103 3.5mm TRS to 1/4" TRS Adaptor ['hosa', 'gpm-103', '3.5mm', 'trs', '1/4"', 'trs', 'adaptor']
Channel Master Titan 2 Antenna Preamplifier ['channel', 'master', 'titan', 'antenna', 'preamplifier']
LIMTECH Wall charger + USB Hotsync & Charging Dock Cradle desktop Charger for Apple IPOD Shuffle 2nd Generation MP3 Player ['limtech', 'wall', 'charger', 'usb', 'hotsync', 'charging', 'dock', 'cradle', 'desktop', 'charger', 'apple', 'ipod', 'shuffle', '2nd', 'generation', 'mp3', 'player']
Skullcandy Air Raid Portable Bluetooth Speaker ['skullcandy', 'air', 'raid', 'portable', 'bluetooth', 'speaker']
Pioneer SP-BS22-LR Andrew Jones Designed Bookshelf Loudspeakers ['pioneer', 'sp-bs22-lr', 'andrew', 'jones', 'designed', 'bookshelf', 'loudspeakers']
C2G/Cables to Go 03170 3.5mm F/F Stereo Coupler ['c2g/cables', 'go', '03170', '3.5mm', 'f

In [7]:
title = df1.product_title.astype('str')
title = title.str.lower()
title = title.drop_duplicates()
title = title.reset_index(drop=True)

In [10]:
print(len(title.unique()))
title.head()

167713


0    yoomall 5m antenna wifi rp-sma female to male ...
1           hosa gpm-103 3.5mm trs to 1/4" trs adaptor
2          channel master titan 2 antenna preamplifier
3    limtech wall charger + usb hotsync & charging ...
4       skullcandy air raid portable bluetooth speaker
Name: product_title, dtype: object

In [11]:
title[162802]

'rca rcr311sn 3 device partially backlit keypad remote (silver) (discontinued by manufacturer)'

In [12]:
len(title)
canonicalize_withoutstem(title[162802])

['rca',
 'rcr311sn',
 'device',
 'partially',
 'backlit',
 'keypad',
 'remote',
 '(silver)',
 '(discontinued',
 'manufacturer)']

In [8]:
title_len = np.zeros(len(title))
for i in range(len(title_len)):
    title_token = canonicalize_withoutstem(title[i])
    title_len[i] = len(title_token)

In [10]:
title_len.mean()

11.027910776147348

In [15]:
title[np.where(title_len<2)[0]]

83                                          no more selling
210                                         sdmx22-004g-par
283                                                    plug
717                                           g.g.martinsen
1149                                        hdmi-high-speed
1223                                                dasteck
1332                                                 g-boom
1463                                          outlets-to-go
1580                                           bluedotsound
1607                                                 kition
2207                                                 viewhd
2333                                              baldeagle
2357                                                   ce-1
2637                                                  grado
2808                                     hb-bt513-headphone
2932                                             headphones
2955                                    

In [11]:
res = es.get(index="test-index", doc_type='tweet', id=656)
print(res['_source'])

{'id': 491993973331075072, 'text': 'Cero hambre ,ni ganas de comer..', 'coordinates': None, 'created_at': 'Wed Jul 23 17:11:18 +0000 2014', 'lang': 'es', 'Timestamp': 0, 'entities': {'hashtags': [], 'urls': [], 'user_mentions': [], 'media': None}, 'place': None, 'user': {'Statuses_count': 9125, 'Name': 'Evoooo', 'Description': 'https://m.facebook.com/evelin.saez', 'Location': '', 'Verified': False, 'Followers_count': 999, 'Profile_image_url_https': 'https://pbs.twimg.com/profile_images/491368555892707328/tCdZk_0a_normal.jpeg', 'Url': '', 'Screen_name': 'evoOsaeZ', 'Friends_count': 1074, 'Created_at': 'Wed Dec 15 01:54:56 +0000 2010', 'Id': 226780037}, 'Retween': False}


In [17]:
len(canonicalize_withoutstem(title[1])[0])
" ".join(canonicalize_withoutstem(title[575]))

'plexdisc 633-814 25 gb 6x blu-ray logo top single layer recordable disc bd-r, 50-disc spindle'

In [18]:
es.indices.refresh(index="test-index")
res = es.search(index="test-index", body={"query": 
                                          {"match":
                                            {"lang": 
                                             {"query": "en"}}}})
print(res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(Timestamp)s %(id)s: %(text)s" % hit["_source"])

10000
0 491993969216061440: RT @TheiPhoneProbz: LOOOL THE FUNNIEST AUTOCORRECTS OF 2014 😂😂😂Check it Out: http://t.co/R97d1zUa3T http://t.co/yZ9qWFUnQF
0 491993969279401984: The way I feel about you, I've never felt with anyone else.
0 491993969178320896: @naturallymcvey I know I was like "uhhhh alright then" haha
0 491993969287761920: RT @biebersmeaning: they're such a cute family. grandpa g was a genuinely smart, caring man. he's in a better place.💖
#RIPGrandpaGrande htt…
0 491993969321320448: RT @ItsMikeyMurphy: @LukeKorns WOW FAKE FAN DIDNT WATCH TO THE END
0 491993969333522433: GovBeat: At least 32 governors have weighed in on the border crisis. Here’s what each has said. http://t.co/DogacpZ7xI
0 491993969165746178: Kent dent Surrey promotion hopes http://t.co/fV5U5X0A7w
0 491993969237041152: @kevinegypt I feel you, just a lot more easier for me to commute!
0 491993969174511616: I only watch Campus PD to laugh at the all the drunk underage kids.
0 491993969249636352: Last gringo an

In [12]:
canonicalize_withoutstem(title[4])

['skullcandy', 'air', 'raid', 'portable', 'bluetooth', 'speaker']

In [77]:
es.indices.refresh(index="test-index")
i = 50
res = es.search(index="test-index", body={"query": {
                                              "bool": {
                                                  "filter": [{"match": {"lang":{"query": "en"}}}],
                                                  "must": [{"match_bool_prefix":{"text":{
                                                    "query": " ".join(canonicalize_withoutstem(title[i])),
                                                     "operator" : "or",
                                                        "minimum_should_match" : 3,}}}
                                                          ,{"match": {"text":{
                                                    "query": canonicalize_withoutstem(title[i])[0],}}}],
                                                         }}})

print(res)
print(res['hits']['total']['value'])
print(canonicalize_withoutstem(title[i]))
for hit in res['hits']['hits']:
    print("%(Timestamp)s %(id)s: %(text)s" % hit["_source"])

{'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 15, 'relation': 'eq'}, 'max_score': 37.572227, 'hits': [{'_index': 'test-index', '_type': 'tweet', '_id': '225014', '_score': 37.572227, '_source': {'id': 491995541983600641, 'text': 'White helmets on display at #Pac12MediaDay http://t.co/OTHPEKJNp7', 'coordinates': None, 'created_at': 'Wed Jul 23 17:17:32 +0000 2014', 'lang': 'en', 'Timestamp': 0, 'entities': {'hashtags': [{'Text': 'Pac12MediaDay', 'Indices': [28, 42]}], 'urls': [], 'user_mentions': [], 'media': [{'Media_url_https': 'https://pbs.twimg.com/media/BtPrVB_CYAESbUN.jpg', 'Media_url': 'http://pbs.twimg.com/media/BtPrVB_CYAESbUN.jpg'}]}, 'place': None, 'user': {'Statuses_count': 26214, 'Name': 'Beto Duran', 'Description': 'Reporter ESPN Radio LA. Lakers TV on Time Warner Cable & Deportes. Announcer Golden Boy Boxing', 'Location': 'Carson', 'Verified': False, 'Followers_count': 22857, 'Profile_image

In [90]:
res['hits']['hits'][6]['_score']

28.905613

In [21]:
for i in range(len(title)):
    print(canonicalize_withoutstem(title[i])[0])

yoomall
hosa
channel
limtech
skullcandy
pioneer
c2g/cables
coolead-hdmi
philips
playstation
jvc
sylvania
coby
diamond
kingvom
jbl
yipbowpt
startech.com
teac
philips
brainwavz
mount
hqrp
anker
seven
cable
monoprice
battery1inc
atlantic
apc
griffin
bose
jbl
over-the-ear
status
fiio
1byone
[new
crosley
cablewholesale
origaudio
sony
tripp
apc
videosecu
panasonic
tripp
husky
pack
headphones,
white
brookstone
twisted
american
lastest
soundxl
sennheiser
comply
videosecu
amazonbasics
g.g.martinsen
ideausa
ac
sony
110v-240v
accell
sony
igadgitz
mohu
soundance®
bluetooth
hdmi
arctic
grace
zipbuds
jbl
koss
mitsubishi
skullcandy
sonos
cable
high
sony
selling
flare
etekcity
jam
ceptics
mount-it!
optical
itis
harman
echogear
aerb®
c&e
1byone
panasonic
omaker
bluedio
cables
playtime,
niles
symphonized
innovative
guppy
sony
tdk
monoprice
po
urge
nitecore
cable
leadtry?
startech.com
orei
tdk
1byone
sony
tfseven
super
philips
audist
v-moda
new
duracell
hitcar
bose
mounting
sennheiser
cyberpower
dknight


directv
definitive
startech
jaras
original
flylion®
jvc
pyle
monster
clarity
ematic
inkjet
philips
memorex
cantilever
new
kabeldirekt
invision
nikon
blue
foxnovo
monster
jvc
plantronics
mediabridge
c&e
2000
bluetooth
pcs
swageu
cables
mitsubishi
allreli
generic
panlong
sonos
khypermedia
cables
nicd
victsing
lutema
cable
granvela
pyle
onh
c&e
yamaha
ilogic
generic
remote
channel
super
sownd
nobsound®
t-power
sanus
aftershokz
cd/dvd
at&t
rca
funai
emazing
neutrik
bose
kollea
bose
tencloud
optimal
onkyo
cable
accessory
full
sony
monster
andrea
update
videosecu
symphonized
monster
mount-it!
sennheiser
isaddle
mitsubishi
etekcity
naxa
riptunes
sangean
digital
niles
port
stinger
ilive
nvx®
technics
yamaha
cerwin-vega
duracell
cable
klipsch
directv
baldeagle
foxnovo
blue
speakercraft
belkin
16
tripp
12v
enjoygadgets
cantilever
sennheiser
atlantic
technics
ge
awo
bose
generic
ibeats
sony
tv
mount-it!
aurabeam
sony
blurex
ce-1
acoustic
rock
panasonic
hometheatre
sony
astron
sylvania
ultra
fiio


IndexError: list index out of range

In [52]:
import time
import numpy as np
es.indices.refresh(index="test-index")
res_n = np.zeros(len(title))
q_len = np.zeros(len(title))
q_first = ""
q = ""
start = time.time()
for i in range(len(title)):
    q = " ".join(canonicalize_withoutstem(title[i]))
    q_len[i] = len(q)
    if (0<len(q)<500):
        q_first = canonicalize_withoutstem(title[i])[0]
        res = es.search(index="test-index", body={"query": {
                                              "bool": {
                                                  "must": [
                                                      {"match_bool_prefix":{"text":{
                                                        "query": q,
                                                        "operater": 'or',
                                                        "minimum_should_match" : 3,}}},
                                                      {"match": {"text":{
                                                        "query": q_first,}}}],
                                                  "must_not": [
                                                      {"match": {"text": {
                                                        "query": 'http://'}}}],
                                                  "filter": [
                                                      {"match": {"lang":{"query": "en"}}}]
                                                  }}})
        res_n[i] = res['hits']['total']['value']
        
end = time.time()
diff = end - start
print("Data processing took %d seconds "% (diff))

Data processing took 517 seconds 


In [57]:
len(np.where(res_n>0)[0])
np.where(res_n>0)[0]

array([    21,     33,     49, ..., 167567, 167574, 167689], dtype=int64)

In [59]:
len(np.where(q_len>500)[0])

8

In [31]:
print(("%d titles are extracted from %d titles ~ %d percents of title.") % (len(np.where(res_n>0)[0]),len(res_n),len(np.where(res_n>0)[0])/len(res_n)*100))

38135 titles are extracted from 167713 titles ~ 22 percents of title.


In [54]:
len(np.where(res_n>0)[0])
for i in np.where(res_n>0)[0]:
    print(title[i])

mount factory articulating swivel full motion tv wall mount bracket for 22-42 inch tv
over-the-ear headphones with volume control
headphones, sound intone i66 metal earphones with microphone, bass earbuds with remote, noise isolating, cute in ear headphones for kids, adults, compatible for iphone, android, ipad, mp3/4
american red cross zoneguard weather radio
bluetooth earphones, taotronics bluetooth wireless earphones sport headphones earbuds headsets (bluetooth 4.0, delicate sound, build-in mic, 6 hours playtime, cvc 6.0 noise-cancelling)
cable management sleeve, joto cable sleeve, [set of 4] 19" neoprene cable sleeves for pc / tv / home theater / speaker, flexible cable wrap, cable cover, cable organizer, cords management (4 piece)
no more selling
flare audio r2a sound isolating in-ear aluminum earphones with high definition & distortion free sound (black)
mount-it! tv wall mount full motion and heavy-duty for samsung, sony, lg, sharp, insignia, vizio, haier, toshiba, sharp, elemen

40, 42, 43 inch tv-protectortm stalysh tv screen protector for lcd, led and plasma tvs
2015 new version jabees sweatproof bluetooth 4.0 sport stereo headphone with apt-x nfc dual microphone for samsung galaxy s6, galaxy s6 edge , iphone 6,6plus, 5s 5c 5 4s 4, ipods,htc one,one mini, one mini 2,ipad mini, samsung galaxy note 3, note 2, s5 s4, s3, s2,lg optimus,lg g3,g2,moto x,most android smart phones and tablets and other bluetooth-enabled tablets
red 3.5mm 1/8"audio cable aux-in cord w mic for monster n-tune on-ear headphone
(with plastic case)digital pocket ph water meter tester for aquarium pool spa wine hydroponics test
cd/dvd blu-ray disc holder with ribbed rubber reinforcement in royal black
new hd dvd player remote se-r0285 se r0285 fit for toshiba hd-a3 hd-a3ku hd-a3kc hd-a30 hd-a30k hd-a3 hd-a3ku hd-a3kc hd-a30 hd-a30ku hd-a30kc
picopsu-120-wi-25 120w wide input dc-dc power converter
t-power 9vdc ( 6.6ft long cable ) ac adapter for m-audio / midiman 9v axiom pro series, axiom 

tv armor tv screen protector that fits 40"-42" lcd, led and plasma tvs
10 pack nitecore i2 intellicharger 2014 version universal home/car battery charger with bonus 6 pack edisonbright aa to d type battery spacer/converters (wholesale)
3) vm audio sreb3 ear earphones earbuds mp3/ipod iphone headphones - white pink
high shield antenna 1ghz splitter/combiner-4-way
mount-it! articulating tv wall mount for medium flat panel tvs suitable for 37-70"
c&e f-pin coaxial quick connect adapter
c&e 288-disc cd/dvd vinyl carrying case
10 pcs sd cf memory card plastic storage jewel case (memory card not included) (2 1/16" x 1 11/16" x 5/16")
triple 3-way n male to two n female rf adapter coaxial cable connector converter
sodial(r) 10 pair deans anti-skid connector t plug male+female for rc esc battery
(1) black 8-disc capacity cd dvd 2-ring album wallet book storage cdbr1608bk (unikeep style)
sodial(r) 2 pcs plated 3.5mm female to double male f/m airplane headphone socket adaptor
monster u3 v600 hdm

new astar all in one triple screen pc / amp-2208,desktop pc
cut-out rose design plastic case for samsung gala
6 small (s) white triple flange replacement eartips earbuds for monster beats dr. dre tour, powerbeats, urbeats 2.0, heartbeats 2.0 (lady gaga), diddybeats and turbine pro, gratitude, dna, diesel vektr, isport victory, isport immersion, inspiration, claritymobile, ncredible n-ergy in-ear stereo earphones
npg18-12 12 volt 18 amph sla replacement battery with nb terminal
new battery replacement for pa3817u-1bas pa3817u-1brs toshiba satellite l700 satellite l730 satellite l735 satellite l740 satellite l745d satellite l745 satellite l755 satellite l755d satellite l770 satellite l770d satellite l775 satellite l775d satellite l750 satellite l750d series
3 packs floureon cordless phone battery 1600mah for ge tl96502, tl86502, tl-86502, tl26502, tl-26502, tl-96502
3.5mm stereo jack(female)to 2.5mm stereo plug(male)adapter
(5) black 12-disc capacity cd dvd 2-ring album wallet book stora

brand new microsoft corporation microsoft flight simulator x deluxe popular high quality
new monster hd cable ultra 800 hdmi to dvi video 16 ft
multi region blu ray dvd player blu ray zone a+b+c, dvd region 0 (1, 2, 3, 4, 5, 6); ntsc/pal, network, usb, hdmi, component video, composite video, coaxial, optical, upscaling, divx avi, divx ultra certified, mkv, xvid avi, vcd, svcd, avchd, music cds
j-tech digital (registered us trademark) premium quality hdmi to hdmi + stereo audio audio extractor converter (rca to spdif)
cheetah mounts professional articulating arm mount for lcd displays/tvs 15"-32" up to 40lbs tilt and swivel
new xgear shadow case for iphone 4 black includes one screen protector carbon fiber leather like
new high quality power acoustik d4 1400 demon series class ab amplifier extruded aluminum chassis remote gain (car stereo amps)
new high quality cobra cpi 1575 3000 watt, 12 volt dc to 120 volt ac power inverter (12 volt-car stereo access)
8" 2-way 250-watt in-ceiling spe

In [163]:
import time
import numpy as np
es.indices.refresh(index="test-index")
# res_n = np.zeros(len(title))
df_tweets = pd.DataFrame( columns = ['Product','Tweets_id','Tweets_create_at','Tweets_text','Tweets_score']) 
q_len = np.zeros(len(title))
q_first = ""
q = ""
start = time.time()
for i in np.where(res_n>0)[0]:
    q = " ".join(canonicalize_withoutstem(title[i]))
    q_len[i] = len(q)
    if (0<len(q)<500):
        q_first = canonicalize_withoutstem(title[i])[0]
        res = es.search(index="test-index", body={"query": {
                                              "bool": {
                                                  "filter": [{"match": {"lang":{"query": "en"}}}],
                                                  "must": [{"match_bool_prefix":{"text":{
                                                    "query": q,
                                                     "operator" : "or",
                                                        "minimum_should_match" : 3,}}}
                                                          ,{"match": {"text":{
                                                    "query": q_first,}}}],
                                                  "must_not": [
                                                      {"match": {"text": {
                                                        "query": 'http://'}}}],
                                                         }}})
#         res_final_n[i] = res_final['hits']['total']['value']
#         print(q)
        if res != None:
            for hit in res['hits']['hits']:
                T_create_at = str(hit["_source"]['created_at'])
                T_id = hit["_source"]['id']
                T_text = str(hit["_source"]['text'])
                T_score = hit['_score']
                df_tweets = df_tweets.append({'Product': q, 'Tweets_id': T_id, 'Tweets_create_at': T_create_at, 'Tweets_text': T_text,'Tweets_score': T_score}, ignore_index=True)

end = time.time()
diff = end - start
print("Data processing took %d seconds "% (diff))

Data processing took 1692 seconds 


In [164]:
df_tweets = df_tweets.drop_duplicates(subset= 'Tweets_text',keep= 'first')

In [165]:
df_tweets.count()
# df_tweets = df_tweets.drop(columns = 'Title')

Product             14748
Tweets_id           14748
Tweets_create_at    14748
Tweets_text         14748
Tweets_score        14748
dtype: int64

In [166]:
df_tweets.to_csv(r'Tweets_targets.csv',index=False)

In [167]:
df_targets = pd.read_csv('Tweets_targets.csv')

In [168]:
df_targets[['Product','Tweets_score']]

,Product,Tweets_score
0,mount factory articulating swivel full motion ...,49.624830
1,mount factory articulating swivel full motion ...,28.007366
2,over-the-ear headphones volume control,25.535366
3,over-the-ear headphones volume control,19.733324
4,over-the-ear headphones volume control,16.820389
5,over-the-ear headphones volume control,15.852055
6,over-the-ear headphones volume control,15.852055
7,over-the-ear headphones volume control,15.323061
8,over-the-ear headphones volume control,15.044106
9,over-the-ear headphones volume control,14.776017


In [0]:
#     print(res['hits']['total']['value'])
# print("Got %d Hits:" % res['hits']['total']['value'])
# for hit in res['hits']['hits']:
#     print("%(Timestamp)s %(id)s: %(text)s" % hit["_source"])

In [0]:
result = np.where(res_n>0)
print(result)
print(title[result[0]])

262            amazonbasics mini portable bluetooth speaker
283                                                    plug
372                      amazonbasics high speed hdmi cable
964       creative sound blaster roar: portable nfc blue...
2038                                          hdmi splitter
2320                                  digital optical audio
2388                    jbl clip portable bluetooth speaker
2637                                                  grado
2932                                             headphones
2955                                              earphones
3132      yamaha rx-v377 5.1-channel a/v home theater re...
3883                                                      1
3909                                             apple ipod
4039                                                    jam
4085                                  high speed hdmi cable
4089                                       apple ipod touch
4194                                    

In [0]:
res = es.search(index="test-index", body={"query": 
                                          {"match_phrase":
                                           {"text":title[result[0][1]]
                                            }}})
print(res['hits']['total']['value'])
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(Timestamp)s %(id)s: %(text)s" % hit["_source"])

89
Got 89 Hits:
0 492005436569030659: Finessing the plug
0 492001866868199424: I lost my plug:(
0 492006212414996480: I lost my plug...
0 491997907525320705: @ThatsSoSchwaby @Mr_Western2014 glow plug system
0 492005063208869888: Must solute the plug ✌️
0 492001183155101697: @bahcarou  Abre o Plug lá mana
0 491994786879270913: I need a sneaker plug ..👀
0 491998347998531586: Plug and play http://t.co/xDp6jl5v1d
0 492005780489785345: @DeezNuts yes. So about this plug?
0 492002756165509120: Gotta find me a plug in TC


In [0]:
sum(res_n)

43953.0